# GOMS 데이터 회귀 분석 - 직업군 예측

In [2]:
import pandas as pd
import numpy as np

In [7]:
data = pd.read_csv('/content/drive/MyDrive/goms/goms_data.csv') # 각자 경로로 바꾸기

In [ ]:
# null 확인
data.isnull().sum()

In [9]:
# 결측치 0으로 대체
data = data.fillna(0)

## 종속변수가 main

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

In [10]:
data_main = data.drop(['schoolType_str','subJobCate', 'mainJobCate_str', 'midJobCate', 'midJobCate_str', 'subJobCate_str'],axis=1)

In [13]:
X_ohe1 = pd.get_dummies(data_main, columns=['major'])
y_main = X_ohe1['mainJobCate']
x_main = X_ohe1.drop(['mainJobCate'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(x_main, y_main, test_size=0.3, random_state=0)
rfr = RandomForestRegressor(max_depth = 3,random_state=12)
rfr.fit(X_train, y_train)

y_train_pred = rfr.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
print("Train MSE: %.4f" % train_mse)
y_test_pred = rfr.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
print("Test MSE: %.4f" % test_mse)

Train MSE: 6.3344
Test MSE: 6.4881


## 종속변수가 mid

In [15]:
data_mid = data.drop(['schoolType_str','subJobCate', 'mainJobCate_str', 'mainJobCate', 'midJobCate_str', 'subJobCate_str'],axis=1)

In [16]:
X_ohe2 = pd.get_dummies(data_mid, columns=['major'])
y_mid = X_ohe2['midJobCate']
x_mid = X_ohe2.drop(['midJobCate'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(x_mid, y_mid, test_size=0.3, random_state=0)
rfr = RandomForestRegressor(max_depth = 3,random_state=12)
rfr.fit(X_train, y_train)

y_train_pred = rfr.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
print("Train MSE: %.4f" % train_mse)
y_test_pred = rfr.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
print("Test MSE: %.4f" % test_mse)

Train MSE: 538.3760
Test MSE: 552.2847
